# Please Find PART 1 and PART 2 of assignment in single notebook

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from bs4 import BeautifulSoup # library to parse html from web page
import requests
from IPython.display import display_html

!conda install -c conda-forge geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Since there was issue to load Geocoder package, i have used mentioned csv file 

In [3]:
data=pd.read_csv('data.csv')
data1=pd.read_csv('http://cocl.us/Geospatial_data')

# DataFrame created earlier with Neighborhood details

In [4]:
data.drop(columns=['Unnamed: 0'],inplace=True)
data.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:

data1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Joining two tables based on Postal code , for this i have first renamed columns in both dataframes inorder to merge them

In [7]:

data1.rename(columns={'Postal Code':'Postalcode'},inplace=True)
data1.head()
data_final=pd.merge(data, data1, on='Postalcode')
#data_final.drop(columns=['Unnamed: 0'],inplace=True)

In [8]:
data_final.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Now we are going to analyse this data based on final data frame created.


## Check the number of boroughs and neighborhoods in Toronto

In [11]:
print('The Toronto dataframe has {} boroughs and {} neighborhoods.'.format(
        len(data_final['Borough'].unique()),
        data_final.shape[0]
    )
)

The Toronto dataframe has 11 boroughs and 103 neighborhoods.


In [12]:
address = 'Toronto, Ontario Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Canada are 43.653963, -79.387207.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_final['Latitude'],data_final['Longitude'], data_final['Borough'], data_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

# For this task, I will just reduce the our target analysis to Neighbohoods in Scarborough only.

In [16]:
Scarborough_data = data_final[data_final['Borough']=='Scarborough']#.reset_index(drop=True)
Scarborough_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Lets lookup with new markers on map

In [81]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [89]:

CLIENT_ID = 'YAO3CRLD5D31AFU3E3O0ZJS0HUY1LVRRLFCW2Z3OAKNUSDBQ' # your Foursquare ID
CLIENT_SECRET = 'DE0F0MFHQD0MBAKKUJJSQNKU1F4TL1E5NARTFE1W30IFWRU4' # your Foursquare Secret
VERSION = '20140715'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
#scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
address1 = 'Scarborough,Toronto'

geolocator1 = Nominatim()
location1 = geolocator1.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude1, longitude1))

Your credentails:
CLIENT_ID: YAO3CRLD5D31AFU3E3O0ZJS0HUY1LVRRLFCW2Z3OAKNUSDBQ
CLIENT_SECRET:DE0F0MFHQD0MBAKKUJJSQNKU1F4TL1E5NARTFE1W30IFWRU4
The geograpical coordinate of Scarborough are 43.773077, -79.257774.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:11: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


Exploring neighborhood using FOURSQUARE API

# Retrieve all Venues in Scarborough neigborhood

In [90]:
neighborhood_latitude = Scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

#print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                              # neighborhood_latitude, 
                                                               #neighborhood_longitude))

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=YAO3CRLD5D31AFU3E3O0ZJS0HUY1LVRRLFCW2Z3OAKNUSDBQ&client_secret=DE0F0MFHQD0MBAKKUJJSQNKU1F4TL1E5NARTFE1W30IFWRU4&ll=43.773077,-79.257774&v=20140715&radius=500&limit=100'

In [100]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [102]:
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(12)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
5,DAVIDsTEA,Tea Room,43.776613,-79.258516
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
8,Hot Topic,Clothing Store,43.775450,-79.257929
9,Tim Hortons,Coffee Shop,43.769936,-79.260368


# Lets check Categories and count of stores by grouping them

In [108]:
nearby_venues.groupby('categories').name.count()

categories
American Restaurant    1
Bakery                 1
Big Box Store          1
Bubble Tea Shop        1
Chinese Restaurant     1
Clothing Store         7
Coffee Shop            3
Cosmetics Shop         2
Department Store       1
Discount Store         1
Electronics Store      1
Fish & Chips Shop      1
Food Court             1
Greek Restaurant       1
Gym                    1
Mexican Restaurant     1
Movie Theater          1
Music Store            1
Pharmacy               2
Pizza Place            1
Restaurant             1
Sandwich Place         2
Shopping Mall          1
Smoothie Shop          1
Sporting Goods Shop    2
Storage Facility       1
Tea Room               2
Toy / Game Store       1
Video Game Store       1
Wings Joint            1
Name: name, dtype: int64

#Clearly above, we can see that there are maximum stores of Clothing in Scarborough neighborhood